In [24]:
import tensorflow as tf
import keras
import keras_vggface
from keras_vggface.vggface import VGGFace
# Had to change code in keras vggface to make it work changed model.py line 20 as shown here https://stackoverflow.com/a/71379206
import numpy as np
import matplotlib as mpl
from keras.utils.data_utils import get_file
import keras_vggface.utils
import PIL
import os
import os.path
import matplotlib.pyplot as plt

In [25]:
train_seed = 42
split = 0.2

train_dataset = keras.utils.image_dataset_from_directory('./Faces', shuffle=True, image_size=(224, 224), batch_size=8, seed=train_seed, validation_split=split, subset='training')
val_dataset = keras.utils.image_dataset_from_directory('./Faces', shuffle=True, image_size=(224, 224), batch_size=8, seed=train_seed, validation_split=split, subset='validation')

Found 300 files belonging to 4 classes.
Using 240 files for training.
Found 300 files belonging to 4 classes.
Using 60 files for validation.


In [26]:
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.2), ])

In [27]:
resnet_base = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))

In [28]:
from keras.layers import Dense, Flatten, Input
from keras_vggface.vggface import VGGFace

num_classes = 4

resnet_base.trainable = False
last_layer = resnet_base.get_layer('avg_pool').output

inputs = tf.keras.Input(shape=(224, 224, 3))

x = data_augmentation(inputs)
x = resnet_base(x)
x = Flatten(name='flatten')(x)

out = Dense(num_classes, name='classifier')(x)

custom_resnet_model = keras.Model(inputs, out)

In [29]:
LR = 0.0001

custom_resnet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [30]:
history = custom_resnet_model.fit(train_dataset, epochs=20, validation_data=val_dataset)

Epoch 1/20


2023-04-30 22:31:05.739322: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


30/30 [==============================] - ETA: 0s - loss: 1.5815 - accuracy: 0.4292

2023-04-30 22:31:09.235602: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


30/30 [==============================] - 7s 131ms/step - loss: 1.5815 - accuracy: 0.4292 - val_loss: 0.4504 - val_accuracy: 0.8167
Epoch 2/20
30/30 [==============================] - 2s 81ms/step - loss: 0.9905 - accuracy: 0.6208 - val_loss: 0.2468 - val_accuracy: 0.9833
Epoch 3/20
30/30 [==============================] - 2s 81ms/step - loss: 0.6354 - accuracy: 0.7792 - val_loss: 0.1926 - val_accuracy: 1.0000
Epoch 4/20
30/30 [==============================] - 2s 81ms/step - loss: 0.7563 - accuracy: 0.7583 - val_loss: 0.1679 - val_accuracy: 1.0000
Epoch 5/20
30/30 [==============================] - 2s 80ms/step - loss: 0.6352 - accuracy: 0.7750 - val_loss: 0.1555 - val_accuracy: 1.0000
Epoch 6/20
30/30 [==============================] - 2s 79ms/step - loss: 0.5206 - accuracy: 0.8250 - val_loss: 0.1432 - val_accuracy: 1.0000
Epoch 7/20
30/30 [==============================] - 2s 79ms/step - loss: 0.4936 - accuracy: 0.8333 - val_loss: 0.1332 - val_accuracy: 1.0000
Epoch 8/20
30/30 [=====

# Privacy

In [31]:
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer
import tensorflow_privacy

In [32]:
train_dataset = keras.utils.image_dataset_from_directory('./Faces', shuffle=True, image_size=(224, 224), batch_size=16, seed=train_seed, validation_split=split, subset='training')
val_dataset = keras.utils.image_dataset_from_directory('./Faces', shuffle=True, image_size=(224, 224), batch_size=16, seed=train_seed, validation_split=split, subset='validation')

Found 300 files belonging to 4 classes.
Using 240 files for training.
Found 300 files belonging to 4 classes.
Using 60 files for validation.


In [33]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction=tf.losses.Reduction.NONE)

In [34]:
noise_multiplier = 1.1
l2_norm_clip = 5
num_microbatches = 16
learning_rate = 0.0001

optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate
)

In [35]:
LR = 0.0001

custom_resnet_model.compile(optimizer=optimizer,
    loss = loss,
    metrics=['accuracy'])

In [36]:
history = custom_resnet_model.fit(train_dataset, epochs=20, validation_data=val_dataset)

Epoch 1/20


2023-04-30 22:31:58.096225: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - ETA: 0s - loss: 6.1115 - accuracy: 0.3667

2023-04-30 22:32:02.200694: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 7s 309ms/step - loss: 6.1115 - accuracy: 0.3667 - val_loss: 9.9964 - val_accuracy: 0.1333
Epoch 2/20
15/15 [==============================] - 3s 206ms/step - loss: 7.8521 - accuracy: 0.2042 - val_loss: 10.3319 - val_accuracy: 0.1333
Epoch 3/20
15/15 [==============================] - 3s 187ms/step - loss: 8.0231 - accuracy: 0.2500 - val_loss: 10.1482 - val_accuracy: 0.1333
Epoch 4/20
15/15 [==============================] - 3s 183ms/step - loss: 8.6566 - accuracy: 0.2625 - val_loss: 10.2234 - val_accuracy: 0.1167
Epoch 5/20
15/15 [==============================] - 3s 181ms/step - loss: 8.4014 - accuracy: 0.2125 - val_loss: 10.6941 - val_accuracy: 0.1167
Epoch 6/20
15/15 [==============================] - 3s 181ms/step - loss: 7.8116 - accuracy: 0.2583 - val_loss: 10.7594 - val_accuracy: 0.1167
Epoch 7/20
15/15 [==============================] - 3s 181ms/step - loss: 8.5918 - accuracy: 0.2708 - val_loss: 10.6923 - val_accuracy: 0.1167
Epoch 8/20
